In [6]:
import fitsio as fio
import numpy as np
import pickle
from tqdm import tqdm
import os, sys

In [14]:
mdet_tilename_filepath = "/global/cfs/cdirs/des/y6-shear-catalogs/Y6A2_METADETECT_V5b/tiles_blinded/mdet_files.txt"
shear_variations_path = "/pscratch/sd/m/myamamot/des-y6-analysis/y6_measurement/v5b_paper/shear_variations_red"
ccd_x_min = 0 # 48
ccd_x_max = 2048 # 2000
ccd_y_min = 0 #48
ccd_y_max = 4096 #4048
cell_side = 128
x_side = int(np.ceil((ccd_x_max - ccd_x_min)/cell_side))
y_side = int(np.ceil((ccd_y_max - ccd_y_min)/cell_side))

In [15]:
def _accum_shear_from_file(ccdres_all, ccdres, x_side, y_side):

    # Create ccd number key. 
    for ccdnum in list(ccdres):
        if ccdnum not in list(ccdres_all):
            ccdres_all[ccdnum] = {}

    cnames = ['g1', 'g2', 'g1p', 'g1m', 'g2p', 'g2m']
    for ccdnum in list(ccdres):
        for cname in cnames:
            if cname not in list(ccdres_all[ccdnum]):
                ccdres_all[ccdnum][cname] = np.zeros((y_side, x_side))
                ccdres_all[ccdnum]["num_" + cname] = np.zeros((y_side, x_side))
            if cname in list(ccdres_all[ccdnum]):
                rows,cols = np.where(~np.isnan(ccdres[ccdnum][cname]))
                np.add.at(ccdres_all[ccdnum][cname], (rows, cols), ccdres[ccdnum][cname][rows, cols])
                np.add.at(ccdres_all[ccdnum]["num_"+cname], (rows, cols), ccdres[ccdnum]["num_"+cname][rows, cols])
                # ccdres_all[ccdnum][cname] += ccdres[ccdnum][cname]
                # ccdres_all[ccdnum]["num_"+cname] += ccdres[ccdnum]["num_"+cname]

    return ccdres_all

In [16]:
mdet_f = open(mdet_tilename_filepath, 'r')
mdet_fs = mdet_f.read().split('\n')[:-1]
mdet_filenames = [fname.split('/')[-1] for fname in mdet_fs]
tilenames = [d.split('_')[0] for d in mdet_filenames]

ccdres_all = {}
for t in tqdm(tilenames):
    with open(os.path.join(shear_variations_path, 'mdet_shear_focal_plane_'+t+'.pickle'), 'rb') as handle:
        ccdres = pickle.load(handle)
    ccdres_all = _accum_shear_from_file(ccdres_all, ccdres, x_side, y_side)
print(list(ccdres_all))

100%|██████████| 10169/10169 [06:51<00:00, 24.68it/s]

[32, 33, 34, 35, 39, 40, 41, 45, 46, 47, 51, 52, 53, 56, 57, 22, 23, 24, 28, 29, 30, 36, 37, 42, 43, 44, 48, 49, 50, 54, 55, 58, 59, 60, 62, 1, 2, 3, 5, 6, 7, 8, 9, 10, 13, 14, 15, 19, 20, 21, 26, 27, 25, 4, 11, 16, 17, 18, 12, 38]


In [17]:
with open(os.path.join(shear_variations_path, 'mdet_shear_focal_plane_all.pickle'), 'wb') as raw:
    pickle.dump(ccdres_all, raw, protocol=pickle.HIGHEST_PROTOCOL)